In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

import windIO
import wind
import windCAD
import foam

## Building Geometry

In [ ]:
caseDir = r"D:\tempData_depot\simData_CandC\wt_tngE2/"

tapFile = caseDir+"tngTapDefinition.csv"
data = pd.read_csv(tapFile)

faceID = [1,2,3,4,5,6]
name = ['Roof_1','Roof_2','Wall_1','Wall_2','Wall_3','Wall_4']
note = ['','','','','','']
origin = [[0,0,0.08],
          [0,-0.0915,0.07809375],
          [-0.1372,0,0],
          [0,-0.0915,0],
          [0.1372,0,0],
          [0,0.0915,0],]
basisVectors = [[[1,0,0],[0,0.999783057,0.020828814],[0,-0.020828814,0.999783057]],
                [[1,0,0],[0,0.999783057,-0.020828814],[0,0.020828814,0.999783057]],
                [[0,0,-1],[-1,0,0],[0,1,0]],
                [[1,0,0],[0,0,-1],[0,1,0]],
                [[0,0,1],[1,0,0],[0,1,0]],
                [[-1,0,0],[0,0,1],[0,1,0]],]
# origin_plt = [[0, 0],
#                 [0, -0.091519855],
#                 [0, -0.2332],
#                 [0, -0.18559375],
#                 [0, -0.2332],
#                 [0, -0.18559375],]
origin_plt = [[0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              ]
# basisVectors_plt = [[[0, -1], [1, 0]],
#                     [[0, -1],[1, 0]],
#                     [[1, 0], [0, 1]],
#                     [[0, -1], [1, 0]],
#                     [[-1, 0], [0, -1]],
#                     [[0, 1], [-1, 0]],]
# basisVectors_plt = [[[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     ]
basisVectors_plt = [[[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    ]
vertices = [[[-0.091519855,-0.1372],[0,-0.1372],[0,0.1372],[-0.091519855,0.1372],[-0.091519855,-0.1372]],
            [[0,-0.1372],[0.091519855,-0.1372],[0.091519855,0.1372],[0,0.1372],[0,-0.1372]],
            [[-0.0915,-0.2332],[0.0915,-0.2332],[0.0915,-0.15510625],[0,-0.1532],[-0.0915,-0.15510625],[-0.0915,-0.2332]],
            [[0.107519855,-0.1372],[0.185613605,-0.1372],[0.185613605,0.1372],[0.107519855,0.1372],[0.107519855,-0.1372]],
            [[-0.0915,0.2332],[0.0915,0.2332],[0.0915,0.15510625],[0,0.1532],[-0.0915,0.15510625],[-0.0915,0.2332]],
            [[-0.107519855,-0.1372],[-0.185613605,-0.1372],[-0.185613605,0.1372],[-0.107519855,0.1372],[-0.107519855,-0.1372]],]

H = 0.08
B = 0.2744
D = 0.183

In [ ]:
# Zone Dict definitions

zoneDict = [
        {    # Face 1
            0:['NBCC', 'Zone c', np.array([[-0.091519855,-0.1372],[-0.071519855,-0.1372],[-0.071519855,-0.1172],[-0.091519855,-0.1172],[-0.091519855,-0.1372]])],
            1:['NBCC', 'Zone c', np.array([[-0.091519855,0.1172],[-0.071519855,0.1172],[-0.071519855,0.1372],[-0.091519855,0.1372],[-0.091519855,0.1172]])],
            2:['NBCC', 'Zone s', np.array([[-0.071519855,-0.1372],[0,-0.1372],[0,-0.1172],[-0.071519855,-0.1172],[-0.071519855,-0.1372]])],
            3:['NBCC', 'Zone s', np.array([[-0.091519855,-0.1172],[-0.071519855,-0.1172],[-0.071519855,0.1172],[-0.091519855,0.1172],[-0.091519855,-0.1172]])],
            4:['NBCC', 'Zone s', np.array([[-0.071519855,0.1172],[0,0.1172],[0,0.1372],[-0.071519855,0.1372],[-0.071519855,0.1172]])],
            5:['NBCC', 'Zone r', np.array([[-0.071519855,-0.1172],[0,-0.1172],[0,0.1172],[-0.071519855,0.1172],[-0.071519855,-0.1172]])],
        },
        {   # Face 2
            0:['NBCC', 'Zone c', np.array([[0.091519855,-0.1372],[0.071519855,-0.1372],[0.071519855,-0.1172],[0.091519855,-0.1172],[0.091519855,-0.1372]])],
            1:['NBCC', 'Zone c', np.array([[0.091519855,0.1172],[0.071519855,0.1172],[0.071519855,0.1372],[0.091519855,0.1372],[0.091519855,0.1172]])],
            2:['NBCC', 'Zone s', np.array([[0.071519855,-0.1372],[0,-0.1372],[0,-0.1172],[0.071519855,-0.1172],[0.071519855,-0.1372]])],
            3:['NBCC', 'Zone s', np.array([[0.091519855,-0.1172],[0.071519855,-0.1172],[0.071519855,0.1172],[0.091519855,0.1172],[0.091519855,-0.1172]])],
            4:['NBCC', 'Zone s', np.array([[0.071519855,0.1172],[0,0.1172],[0,0.1372],[0.071519855,0.1372],[0.071519855,0.1172]])],
            5:['NBCC', 'Zone r', np.array([[0.071519855,-0.1172],[0,-0.1172],[0,0.1172],[0.071519855,0.1172],[0.071519855,-0.1172]])],
        },
        {   # Face 3
            0:['NBCC', 'Zone e', np.array([[-0.0915,-0.2332],[-0.0715,-0.2332],[-0.0715,-0.154689583],[-0.0915,-0.15510625],[-0.0915,-0.2332]])],
            1:['NBCC', 'Zone e', np.array([[0.0715,-0.2332],[0.0915,-0.2332],[0.0915,-0.15510625],[0.0715,-0.154689583],[0.0715,-0.2332]])],
            2:['NBCC', 'Zone w', np.array([[-0.0715,-0.2332],[0.0715,-0.2332],[0.0715,-0.154689583],[0,-0.1532],[-0.0715,-0.154689583],[-0.0715,-0.2332]])],
        },
        {   # Face 4
            0:['NBCC', 'Zone e', np.array([[0.107519855,-0.1372],[0.185613605,-0.1372],[0.185613605,-0.1172],[0.107519855,-0.1172],[0.107519855,-0.1372]])],
            1:['NBCC', 'Zone e', np.array([[0.107519855,0.1172],[0.185613605,0.1172],[0.185613605,0.1372],[0.107519855,0.1372],[0.107519855,0.1172]])],
            2:['NBCC', 'Zone w', np.array([[0.107519855,-0.1172],[0.185613605,-0.1172],[0.185613605,0.1172],[0.107519855,0.1172],[0.107519855,-0.1172]])],
        },
        {   # Face 5
            0:['NBCC', 'Zone e', np.array([[-0.0915,0.2332],[-0.0715,0.2332],[-0.0715,0.154689583],[-0.0915,0.15510625],[-0.0915,0.2332]])],
            1:['NBCC', 'Zone e', np.array([[0.0715,0.2332],[0.0915,0.2332],[0.0915,0.15510625],[0.0715,0.154689583],[0.0715,0.2332]])],
            2:['NBCC', 'Zone w', np.array([[-0.0715,0.2332],[0.0715,0.2332],[0.0715,0.154689583],[0,0.1532],[-0.0715,0.154689583],[-0.0715,0.2332]])],
        },
        {   # Face 6
            0:['NBCC', 'Zone e', np.array([[-0.107519855,-0.1372],[-0.185613605,-0.1372],[-0.185613605,-0.1172],[-0.107519855,-0.1172],[-0.107519855,-0.1372]])],
            1:['NBCC', 'Zone e', np.array([[-0.107519855,0.1172],[-0.185613605,0.1172],[-0.185613605,0.1372],[-0.107519855,0.1372],[-0.107519855,0.1172]])],
            2:['NBCC', 'Zone w', np.array([[-0.107519855,-0.1172],[-0.185613605,-0.1172],[-0.185613605,0.1172],[-0.107519855,0.1172],[-0.107519855,-0.1172]])],
        },
]

# file = caseDir+'NBCC_zoneDict.json'
# with open(file, 'w') as f:
#     json.dump(zoneDict,f, indent=4, separators=(',', ':'))

In [ ]:

faces = []
for i,f in enumerate(faceID):
    idx = data.index[data.faceID == f]
    tapCoords = np.transpose(np.array([data.x[idx], data.y[idx]]))
    tapNos = np.array(data.tapNo[idx],dtype=int)
    idxOrig = idx

    fc = windCAD.face(
                name=name[i],
                ID=f,
                origin=origin[i],
                basisVectors=basisVectors[i],
                origin_plt=origin_plt[i],
                basisVectors_plt=basisVectors_plt[i],
                vertices=vertices[i],
                tapNo=tapNos,
                tapIdx=idxOrig,
                tapName=None,
                tapCoord=tapCoords,
                zoneDict=zoneDict[i],
                # nominalPanelAreas=[1.0e-4, 2.45e-4, 4.8e-4, 7.1e-4, 1e-3], #[5e-5, 8e-5, 1.5e-4, 5e-4],
                nominalPanelAreas=[4.4e-5, 8.15e-5, 1.3e-4, 5e-4, 6.8e-4, 1.3e-3,],   # Final! All are tested. 1.3e-3 results in blunt edge zones in the walls but no error.
                # nominalPanelAreas=[1.3e-3,], 
                # nominalPanelAreas=[4.8e-4, 7.2e-4, 1e-3],
                numOfNominalPanelAreas=3,
                )
    # fc.plot(figSize=[20,15], overlayPanels=True, overlayTaps=True, overlayTribs=True, overlayZones=True)
    faces.append(fc)

# file = caseDir+'ttu_bldgGeom.json'
# allFaces.writeToFile(file_basic=file)

In [ ]:
ttu = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='TNG-E2R1',
                )
print(ttu.error_in_panels)
print(ttu.error_in_zones)

In [ ]:
fig = plt.figure(figsize=[10,8])
ax = fig.add_subplot()

ttu.plotEdges(ax=ax)
ttu.plotZones(ax=ax)
ttu.plotTaps(ax=ax)
ttu.plotPanels(ax=ax,aIdx=5)

ax.axis('equal')
ax.axis('off')



## Wind field

In [ ]:
dt = 3.7890e-04

file = r'D:\tempData_depot\simData_CandC\tngE2\tngE2Fr1_profile.csv'
temp = pd.read_csv(file)

file = caseDir+'tngE2Fr1_TH-UofT.npy'
U_TH = np.load(file)
file = caseDir+'tngE2Fr1_TH-VofT.npy'
V_TH = np.load(file)
file = caseDir+'tngE2Fr1_TH-WofT.npy'
W_TH = np.load(file)


In [ ]:

vel = wind.profile(name="BLWT-E2", Z=temp.Z,UofT=U_TH,VofT=V_TH,WofT=W_TH,H=H,dt=dt,nSpectAvg=64)
vel.plot()

## BLWT $C_p$ data

In [ ]:
mainRefPitotChnlIdx = 2
fps2mps = 0.3048
Zpitot = 1.48
Ntaps = 456

dataDirHFPI = r'E:\TNG\rawHFPI\tngE2p1/'

N_AoA = 3
# AoAids = [str(i).zfill(3) for i in range(1,N_AoA+1)]
AoAids = ['001', '010', '019']

AoA = np.zeros((N_AoA))
sampleRate = np.zeros((N_AoA))

for i,a in enumerate(AoAids):
    file_pssd = dataDirHFPI+'TNGp1E02R001P'+ AoAids[i] +'a.pssd'
    file_pssr = dataDirHFPI+'TNGp1E02R001P'+ AoAids[i] +'a.pssr'

    cp_data,analog,WTTDATALOG = windIO.readPSSfile(file_pssr,file_pssd)

    if i == 0:
        N_t = np.shape(cp_data)[0]
        CpTH = np.zeros((N_AoA,Ntaps,N_t)) # [N_AoA,Ntaps,Ntime]
        UpitotOfT = np.zeros((N_AoA,N_t))
    CpTH[i,:,:] = np.transpose(cp_data[:,0:Ntaps])
    UpitotOfT[i,:] = 29.917 * np.sqrt(analog[:,mainRefPitotChnlIdx]) * fps2mps
    AoA[i] = np.round(WTTDATALOG["APPSPE"][0][0][0][0][0][0][0][16][0][0],1)
    sampleRate[i] = WTTDATALOG["APPSPE"][0][0][0][0][0][0][0][12][0][0]
Upitot = np.mean(UpitotOfT,axis=1)



In [ ]:
ttu = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='TNG-E2R1',
                refProfile=vel,
                Zref_input=Zpitot,  # for the Cp TH being input below
                Uref_input=Upitot,  # for the Cp TH being input below
                samplingFreq=sampleRate[0],
                airDensity=1.125,
                AoA=AoA,
                CpOfT=CpTH,  # Cp TH referenced to Uref at Zref
                badTaps=None, # tap numbers to remove
                reReferenceCpToH=True, # whether or not to re-reference Cp building height
                pOfT=None,
                p0ofT=None,
                CpStats=None,
                peakMethod='minmax',
                )


In [ ]:
fig = plt.figure(figsize=[20,15])
ax = fig.add_subplot()
ttu.plotZones(ax=ax)
ax.axis('equal')

In [ ]:
f = 1
fc = ttu[f]
a = 3
d = 1
fldRange=[-2,1]

fig = plt.figure(figsize=[20,15])
ax = fig.add_subplot()
ttu.plotPanelCpStatContour(fieldName='mean',dxnIdx=d,aIdx=a,showValueText=False,ax=ax,fldRange=fldRange)
ttu.plotPanels(ax=ax,aIdx=a)
ttu.plotEdges(ax=ax,showName=False)
ttu.plotZones(ax=ax)
# ttu.plotTaps(ax=ax,showTapNo=True,fontsize=10)
# ttu.plotTribs(ax=ax)
ax.axis('equal')


In [ ]:
# fld = 'mean'
# # print(ttu.error_in_panels)
# # print(ttu.error_in_zones)
# for i in range(5):
#     print(f"A: {np.shape(ttu.panelAreas[i][2])}, \tGCp:{np.shape(ttu.CpStatsAreaAvgByZone[i][2]['mean'])}")
# for f,fc in enumerate(ttu):
#     for z,zone in enumerate(fc.zoneDict):
#         for a, area in enumerate(fc.nominalPanelAreas):
#             print(f"Zone: {fc.zoneDict[zone][0:2]}\tAreaIdx:{a}\tNo. panels: {len(fc.panels[z][a].geoms)}\tPanel areas: {len(fc.panelAreas[z][a])}\t Weights: {len(fc.tapWghtPerPanel[z][a])} \tCpAvg.: {np.shape(ttu.CpStatsAreaAvg[f][z][a][fld])}")
#             if not len(fc.panelAreas[z][a]) == np.shape(ttu.CpStatsAreaAvg[f][z][a][fld])[1]:
#                 print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:

# for z in ttu.panelAreas:
#     print(ttu.panelAreas[z][0:2])

#     plt.figure()
#     plt.semilogx(ttu.panelAreas[z][2], ttu.CpStatsAreaAvgByZone[z][2]['peakMin'][0,:],'.k')
#     plt.show()

# LES

In [ ]:
lesCase = r'D:\tempData_depot\simData_CandC\ttuE087_900.0/'

## Wind field

In [ ]:

lesProf = foam.processVelProfile(caseDir=lesCase, probeName='probes.V1',H=H, trimTimeSegs=[[0,1.0]])



In [ ]:
p0Idx = 30
_, _, p0OfT = foam.readProbe(probeName='probes.V0', postProcDir=lesCase+'postProcessing/', field='p', trimTimeSegs=[[0,1.0]],)

pOfTfile = lesCase+'postProcessing/wallPressure_pOfT_TNGtaps.npy'
pOfT = np.load(pOfTfile)
m,n = np.shape(pOfT)
pOfT = np.reshape(pOfT,(1,m,n))
print(np.shape(pOfT))

p0OfT = np.reshape(np.transpose(p0OfT)[p0Idx,:n],(1,-1))
print(np.shape(p0OfT))


In [ ]:



ttuLES = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='LES-E087-90deg',
                refProfile=lesProf,
                Zref_input=H,  # for the Cp TH being input below
                Uref_input=lesProf.Uh,  # for the Cp TH being input below
                samplingFreq=lesProf.samplingFreq,
                airDensity=1.125,
                AoA=[90.0,],
                # CpOfT=CpTH,  # Cp TH referenced to Uref at Zref
                # badTaps=None, # tap numbers to remove
                # reReferenceCpToH=True, # whether or not to re-reference Cp building height
                pOfT=pOfT,
                p0ofT=p0OfT,
                # CpStats=None,
                peakMethod='minmax',
                )

In [ ]:
f = 1
fc = ttu[f]
a = 3
d = 2
fldRange=[-2,1]

fig = plt.figure(figsize=[10,7.5])
ax = fig.add_subplot()
ttu.plotPanelCpStatContour(fieldName='mean',dxnIdx=d,aIdx=a,showValueText=False,ax=ax,fldRange=fldRange)
# ttu.plotPanels(ax=ax,aIdx=a)
# ttu.plotTapField(ax=ax,field='mean')
ttu.plotEdges(ax=ax,showName=False)
ttu.plotZones(ax=ax)
# ttu.plotTaps(ax=ax,showTapNo=True,fontsize=10)
# ttu.plotTribs(ax=ax)
ax.set_title('BLWT')
ax.axis('equal')

d = 0
fig = plt.figure(figsize=[10,7.5])
ax = fig.add_subplot()
ttuLES.plotPanelCpStatContour(fieldName='mean',dxnIdx=d,aIdx=a,showValueText=False,ax=ax,fldRange=fldRange)
# ttuLES.plotPanels(ax=ax,aIdx=a)
# ttuLES.plotTapField(ax=ax,field='mean')
ttuLES.plotEdges(ax=ax,showName=False)
ttuLES.plotZones(ax=ax)
# ttuLES.plotTaps(ax=ax,showTapNo=True,fontsize=10)
# ttuLES.plotTribs(ax=ax)
ax.set_title('LES')
ax.axis('equal')

In [ ]:
print((ttu.CpStats['std']))

# CAD lab

In [ ]:
import shapely.geometry as shp
from shapely.ops import voronoi_diagram
from shapely.validation import make_valid
from typing import List, Tuple, Dict

from scipy.spatial import Voronoi, voronoi_plot_2d

def meshRegionWithPanels(region,area,minAreaFactor=0.5,debugMode=False) -> Tuple[shp.MultiPolygon, List[float]]:
    if debugMode:
        print(f"Region: shape {region.shape}, {region}\n")
        print(f"NominalArea = {area}")

    d = np.sqrt(area)
    # generate a uniform grid of panel center points covering the extents of the region
    xMin, xMax = min(region[:,0]), max(region[:,0])
    yMin, yMax = min(region[:,1]), max(region[:,1])
    Dx = xMax-xMin
    Dy = yMax-yMin
    N = int(np.ceil(Dx/d))  # max(int(np.round(Dx/d,0)), 1)
    dx = Dx/N
    M = int(np.ceil(Dy/d))
    dy = Dy/M

    x = np.linspace(xMin+dx/2,xMax-dx/2,N) # np.linspace(xMin+dx/2,xMax-dx/2,N-1) if N > 2 else (xMax-xMin)/2.0
    y = np.linspace(yMin+dy/2,yMax-dy/2,M) # if M > 2 else (yMax-yMin)/2.0
    # x = np.mean([xMin,xMax]) if N == 1 else np.linspace(xMin+dx/2,xMax-dx/2,N) 
    # y = np.mean([yMin,yMax]) if M == 1 else np.linspace(yMin+dy/2,yMax-dy/2,M) 
    X,Y = np.meshgrid(x,y)
    XY = np.concatenate((np.reshape(X,[-1,1]), np.reshape(Y,[-1,1])),axis=1)
    isSinglePoint = len(X) == 1 and len(Y) == 1

    if debugMode:
        print(f"x Range: {xMin} to {xMax} \t\t y Range: {yMin} to {yMax}")
        print(f"dx = {dx}, dy = {dy}")
        print(f"Point grid: {N} X {M}")
        print(f"Grid \t\tx: {x}\n\t\ty: {y} ")
        print(f"Grid mesh shape: {np.shape(X)}")

    # generate the panels with the regular points. Take a note of the panels with area less than minAreaFactor*targetPanelArea for merging
    points1 = shp.MultiPoint(XY)
    regions1 = voronoi_diagram(points1)
    bound = shp.Polygon(region)

    if debugMode:
        print(f"Generated bulk voronoi polygons: {len(regions1.geoms)}")
        plt.figure(figsize=[15,10])
        x,y = bound.exterior.xy
        plt.plot(x,y,'-b',lw=3)
        # plt.plot(X,Y,'.r')
        plt.axis('equal')

    panels1 = []
    goodPts = []
    for g in regions1.geoms:
        newRig = windCAD.getIntersection(bound,g)
        if newRig is not None:
            panels1.append(newRig)
        else:
            continue
        if newRig.area > minAreaFactor*area:
            center = newRig.centroid
            goodPts.append(center.xy)
        if debugMode:
            x,y = newRig.exterior.xy
            # plt.fill(x,y,'y',alpha=0.1)
            # plt.plot(x,y,'-b',alpha=1,lw=0.5)
    goodPts = np.reshape(np.asarray(goodPts,dtype=float),[-1,2])
    if debugMode:
        print(f"Original grid points: {len(panels1)}")
        print(f"Picked good grid points: {len(goodPts)}")
        
    # regenerate the panels without those that had areas less than minAreaFactor*targetPanelArea
    points = shp.MultiPoint(goodPts)
    regions = voronoi_diagram(points)
    
    if debugMode:
        # reg = Voronoi(goodPts)
        # xy = reg.ridge_points
        # plt.plot(xy[:,0],xy[:,1],'.g')
        plt.plot(goodPts[:,0],goodPts[:,1],'.g')
        # for g in regions.geoms:
        #     x,y = g.exterior.xy
        #     plt.plot(x,y,'r')

    panels = []
    areas = []
    for i,g in enumerate(regions.geoms):
        newRig = windCAD.getIntersection(bound,g)
        if newRig is not None:
            panels.append(newRig)
        else:
            continue
        areas.append(newRig.area)
        x,y = newRig.exterior.xy
        if debugMode:
            plt.fill(x,y,'r',alpha=0.5)
            plt.plot(x,y,'--k',alpha=1,lw=1)
            plt.text(np.mean(x),np.mean(y),str(i))

    if len(panels) == 0 or isSinglePoint:
        panels.append(bound)
        areas = [bound.area,]
        if debugMode:
            x,y = bound.exterior.xy
            plt.fill(x,y,'r',alpha=0.5)
            plt.plot(x,y,'--k',alpha=1,lw=1)
    if debugMode:
        print(f"Zone area = {bound.area}, summed area = {np.sum(areas)}")
        plt.axis('equal')
        plt.show()
 
    panels = shp.MultiPolygon(panels)
    return panels, areas

# f,a,z = 4,2,2
f,a,z = 0,0,4
polygon = ttu[f].zoneDict[z][2]
A = 1.2e-3 #ttu[f].nominalPanelAreas[a]+0.000005
print(polygon)

panels, areas = meshRegionWithPanels(region=polygon, area=A, debugMode=True)

In [ ]:

print(ttu.NumPanelsPerArea)
print(ttu.NumPanels)

# fig = plt.figure(figsize=[30,20])
# ax = fig.add_subplot()

# ttu.plotEdges(ax=ax)
# ttu.plotZones(ax=ax)
# ttu.plotTaps(ax=ax)
# ttu.plotPanels(ax=ax,aIdx=a)

# ax.axis('equal')
# ax.axis('off')

# Plot Lab

In [ ]:
print(np.shape(ttu.CpStats['mean']))

In [ ]:

ttu.plotTapCpStatsPerAoA(fldRange=[-8,2.3],xticks=np.round(ttu.AoA,0),pageNo_xy=(0.5,0.06),nCols=13, figsize=[25,20])